In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/sample_submission.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=8/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
/kaggle/input/jane-street-real-time-market

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd


from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LassoCV
from sklearn.linear_model import lasso_path

from sklearn.feature_selection import f_regression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [3]:
# Initialize a list to hold samples from each file
samples = []
# Load a sample from each file
for i in range(10):
    file_path = f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
    chunk = pd.read_parquet(file_path)
    
    # Take a sample of the data (adjust sample size as needed)
    sample_chunk = chunk.sample(n=400000, random_state=42)  # For example, 100 rows
    # sample_chunk = chunk.sample(n=5000, random_state=42)  # For example, 100 rows
    samples.append(sample_chunk)

# Concatenate all samples into one DataFrame if needed
sample_df = pd.concat(samples, ignore_index=True)

In [4]:
sample_df

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,85,288,38,3.119714,NaN,NaN,NaN,NaN,NaN,0.188973,...,-0.191460,0.386567,-0.180631,-0.394700,0.212311,1.695289,-0.002445,-0.041428,0.147673,0.352121
1,83,69,0,2.372535,NaN,NaN,NaN,NaN,NaN,-0.249393,...,-0.304067,2.162238,0.687817,0.344059,2.191972,0.183375,0.511344,1.261587,-0.337162,0.283835
2,82,199,16,1.943764,NaN,NaN,NaN,NaN,NaN,-0.034782,...,-0.174753,-0.093317,-0.030093,-0.060351,-0.016916,0.141347,0.084314,0.035796,0.130798,0.339303
3,0,779,9,2.285698,NaN,NaN,NaN,NaN,NaN,-0.200830,...,-0.264010,-0.010857,-0.047279,0.000333,0.223133,0.273011,0.259631,0.265585,0.200747,-0.067575
4,169,178,7,1.789037,NaN,NaN,NaN,NaN,NaN,0.695718,...,-0.256375,-0.107448,-0.852934,-0.150064,-0.000850,-0.035807,-0.328767,0.106589,0.703084,-0.247847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999995,1568,337,22,1.483402,-0.250877,0.703972,-0.622578,-0.812593,0.198337,-0.097572,...,0.425949,-0.161849,-0.199491,-0.186034,0.039603,0.220486,0.069445,0.179382,0.544421,0.178335
3999996,1689,731,34,2.610843,3.598831,0.408036,2.991286,3.461944,0.630740,0.406703,...,-0.219123,0.007456,0.062822,0.110861,0.443212,0.994053,0.316815,0.439490,0.897641,0.446663
3999997,1569,891,0,4.485613,-1.190200,-1.158374,-0.981779,-1.110746,0.127142,-0.639754,...,-0.219195,-0.027795,0.348221,0.060387,0.260555,-0.608434,-0.153819,0.486034,-0.871301,-0.371311
3999998,1621,673,27,2.641389,1.090095,-0.075986,0.656529,1.341315,1.743172,0.304291,...,-0.672680,0.064354,0.182041,0.640435,1.956136,1.837224,-0.522151,1.555422,1.732854,-0.910796


In [5]:
# 將 sample_df 按照 date 與 time 排序
sample_df = sample_df.sort_values(by=['date_id', 'time_id']).reset_index(drop=True)
sample_df

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
1,0,1,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.574611,...,-0.265587,0.249527,0.324737,0.894854,2.433310,1.498122,2.685856,3.405593,1.713544,3.902426
2,0,2,14,0.440570,NaN,NaN,NaN,NaN,NaN,1.306166,...,2.739192,-0.217807,-0.364068,0.370327,-2.549187,-0.842242,-5.000000,-2.704485,-1.017690,-5.000000
3,0,2,33,1.663408,NaN,NaN,NaN,NaN,NaN,1.125607,...,-0.259715,0.182786,0.196892,-0.461171,1.718243,-0.068831,-0.007666,2.112768,-0.385840,0.531010
4,0,3,16,1.118269,NaN,NaN,NaN,NaN,NaN,0.984437,...,-0.260452,1.078218,0.801424,-0.202072,1.398661,-0.186301,-0.037735,0.731364,0.291155,0.305233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999995,1698,965,30,1.853885,2.814784,-1.028521,2.323651,2.228162,0.946392,0.237958,...,-0.227781,-0.260602,0.138897,-0.405433,-0.314154,-0.195008,-0.037747,-0.136817,-0.019973,-0.214061
3999996,1698,966,12,2.904539,2.626125,-0.359452,2.648303,1.961856,0.114874,0.474699,...,-0.157781,-0.381249,-0.011701,0.923845,-0.464887,-0.278194,-0.436025,-0.807103,-0.261686,-0.699192
3999997,1698,967,5,3.261852,2.684347,-0.872678,2.311741,2.779893,0.140192,0.686878,...,3.959293,0.478624,1.035064,1.767671,0.071564,0.030428,0.109278,-0.065540,-0.012579,-0.168721
3999998,1698,967,11,1.967007,2.557203,-1.061826,2.127367,2.565700,0.182079,0.891533,...,-0.206792,2.384646,0.797282,0.454187,1.705050,0.695965,0.161474,-0.004736,0.024396,-0.024785


In [6]:
# Separate features and responders
features = sample_df.filter(regex='^feature_')
responders = sample_df.filter(regex='^responder_')
features

,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,...,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
0,NaN,NaN,NaN,NaN,NaN,1.139366,0.273328,0.306549,-1.262223,42,...,-0.206929,-0.530602,4.765215,0.571554,NaN,NaN,-0.226891,-0.251412,-0.215522,-0.296244
1,NaN,NaN,NaN,NaN,NaN,1.574611,-0.463503,0.057437,-0.797103,42,...,-0.246006,-0.498534,5.864668,1.699041,NaN,NaN,-0.240066,-0.259317,-0.288024,-0.265587
2,NaN,NaN,NaN,NaN,NaN,1.306166,0.294545,0.298191,-0.807842,44,...,-0.642851,-1.307235,0.031796,-0.569148,NaN,NaN,1.018748,1.476322,2.258740,2.739192
3,NaN,NaN,NaN,NaN,NaN,1.125607,0.211509,0.194039,-0.788869,11,...,-0.629582,-1.246022,-0.171890,-0.578686,NaN,NaN,-0.265635,-0.326711,-0.380055,-0.259715
4,NaN,NaN,NaN,NaN,NaN,0.984437,0.266995,0.298057,-0.783055,11,...,-0.326494,-1.135596,-0.028314,-0.798880,NaN,NaN,-0.248241,-0.325360,-0.283683,-0.260452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999995,2.814784,-1.028521,2.323651,2.228162,0.946392,0.237958,0.809954,1.172599,0.183669,81,...,-0.395111,-0.456819,-0.002942,-0.253851,-0.315238,-0.291550,-0.219557,-0.194489,-0.269783,-0.227781
3999996,2.626125,-0.359452,2.648303,1.961856,0.114874,0.474699,0.651758,1.613531,0.261012,70,...,0.142330,0.101429,0.807534,0.539511,-0.279380,-0.258512,0.064861,0.089479,-0.245598,-0.157781
3999997,2.684347,-0.872678,2.311741,2.779893,0.140192,0.686878,1.602279,2.404034,0.296327,2,...,0.129773,0.480861,0.397526,0.567916,5.272311,3.405017,3.913973,3.298062,4.096728,3.959293
3999998,2.557203,-1.061826,2.127367,2.565700,0.182079,0.891533,1.759385,2.253046,0.338756,46,...,0.274677,1.416519,0.626940,1.158743,-0.350792,-0.241679,-0.229022,-0.274400,-0.285774,-0.206792


## 處理 Lag

In [7]:
# 將 sample_df 的 weight, feature_ 欄位去除
resp = sample_df.drop(columns=['weight'])
resp = resp.drop(columns=features.columns)

# 將 resp date_id 欄位-1
resp['date_id'] = resp['date_id'] - 1
resp



,date_id,time_id,symbol_id,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,-1,0,10,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
1,-1,1,10,0.249527,0.324737,0.894854,2.433310,1.498122,2.685856,3.405593,1.713544,3.902426
2,-1,2,14,-0.217807,-0.364068,0.370327,-2.549187,-0.842242,-5.000000,-2.704485,-1.017690,-5.000000
3,-1,2,33,0.182786,0.196892,-0.461171,1.718243,-0.068831,-0.007666,2.112768,-0.385840,0.531010
4,-1,3,16,1.078218,0.801424,-0.202072,1.398661,-0.186301,-0.037735,0.731364,0.291155,0.305233
...,...,...,...,...,...,...,...,...,...,...,...,...
3999995,1697,965,30,-0.260602,0.138897,-0.405433,-0.314154,-0.195008,-0.037747,-0.136817,-0.019973,-0.214061
3999996,1697,966,12,-0.381249,-0.011701,0.923845,-0.464887,-0.278194,-0.436025,-0.807103,-0.261686,-0.699192
3999997,1697,967,5,0.478624,1.035064,1.767671,0.071564,0.030428,0.109278,-0.065540,-0.012579,-0.168721
3999998,1697,967,11,2.384646,0.797282,0.454187,1.705050,0.695965,0.161474,-0.004736,0.024396,-0.024785


In [8]:
features_df = sample_df.drop(columns=responders.columns)
features_df["index"] = features_df.index
features_df 

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,index
0,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,-0.530602,4.765215,0.571554,NaN,NaN,-0.226891,-0.251412,-0.215522,-0.296244,0
1,0,1,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.574611,...,-0.498534,5.864668,1.699041,NaN,NaN,-0.240066,-0.259317,-0.288024,-0.265587,1
2,0,2,14,0.440570,NaN,NaN,NaN,NaN,NaN,1.306166,...,-1.307235,0.031796,-0.569148,NaN,NaN,1.018748,1.476322,2.258740,2.739192,2
3,0,2,33,1.663408,NaN,NaN,NaN,NaN,NaN,1.125607,...,-1.246022,-0.171890,-0.578686,NaN,NaN,-0.265635,-0.326711,-0.380055,-0.259715,3
4,0,3,16,1.118269,NaN,NaN,NaN,NaN,NaN,0.984437,...,-1.135596,-0.028314,-0.798880,NaN,NaN,-0.248241,-0.325360,-0.283683,-0.260452,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999995,1698,965,30,1.853885,2.814784,-1.028521,2.323651,2.228162,0.946392,0.237958,...,-0.456819,-0.002942,-0.253851,-0.315238,-0.291550,-0.219557,-0.194489,-0.269783,-0.227781,3999995
3999996,1698,966,12,2.904539,2.626125,-0.359452,2.648303,1.961856,0.114874,0.474699,...,0.101429,0.807534,0.539511,-0.279380,-0.258512,0.064861,0.089479,-0.245598,-0.157781,3999996
3999997,1698,967,5,3.261852,2.684347,-0.872678,2.311741,2.779893,0.140192,0.686878,...,0.480861,0.397526,0.567916,5.272311,3.405017,3.913973,3.298062,4.096728,3.959293,3999997
3999998,1698,967,11,1.967007,2.557203,-1.061826,2.127367,2.565700,0.182079,0.891533,...,1.416519,0.626940,1.158743,-0.350792,-0.241679,-0.229022,-0.274400,-0.285774,-0.206792,3999998


In [9]:
# 將 features, responders(前一天的) 合併
merged_df = pd.merge(features_df, resp, on=['date_id', 'time_id', 'symbol_id'], how='inner')
df = merged_df.drop(columns=['date_id', 'time_id'])
inner_index = df['index']
df = df.drop(columns=['index'])

In [10]:
# 將 symbol_id 轉換成 0, 1, getdummies
symbol_id = df['symbol_id']
symbol_id_one_hot = pd.get_dummies(symbol_id, prefix='symbol_id')

# 合併 symbol_id_one_hot 到 df
df = pd.concat([df, symbol_id_one_hot], axis=1)
df = df.drop(columns=['symbol_id'])
df = df.replace({True: 1, False: 0})
df

,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,...,symbol_id_29,symbol_id_30,symbol_id_31,symbol_id_32,symbol_id_33,symbol_id_34,symbol_id_35,symbol_id_36,symbol_id_37,symbol_id_38
0,1.370613,NaN,NaN,NaN,NaN,NaN,0.416936,0.320742,0.215326,-0.601638,...,0,0,0,0,0,0,0,0,0,0
1,2.456331,NaN,NaN,NaN,NaN,NaN,0.793957,0.229449,0.210990,-0.646882,...,0,0,0,0,0,0,0,0,0,0
2,3.889038,NaN,NaN,NaN,NaN,NaN,0.950194,-0.068810,0.152106,-0.977214,...,0,0,0,0,0,0,0,0,0,0
3,1.663408,NaN,NaN,NaN,NaN,NaN,1.193287,0.171371,0.195073,-0.885036,...,0,0,0,0,1,0,0,0,0,0
4,2.456331,NaN,NaN,NaN,NaN,NaN,0.694218,-0.429424,-0.114676,-0.530343,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382089,0.886913,1.959258,0.159797,1.665929,2.188920,-0.211744,-0.523108,-2.251404,-1.161833,0.074578,...,0,0,0,0,0,0,0,1,0,0
382090,2.239790,2.315498,-0.094879,2.199011,2.242779,-0.518142,-0.381145,-2.891396,-1.261238,0.075932,...,0,0,0,0,0,0,0,0,0,0
382091,1.596111,1.702054,-0.874552,2.361620,2.212176,-0.648974,-0.244236,-0.332836,-0.257594,0.026408,...,0,0,0,0,0,0,0,0,0,0
382092,3.009663,2.213526,-0.562913,1.868213,1.911354,-0.398528,-0.140718,0.640867,-0.100757,0.025534,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 將 df 標準化 minmax   
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaler = scaler.fit_transform(df)
df_scaler = pd.DataFrame(df_scaler)
df_scaler

# scaler = StandardScaler()
# df_scaler = scaler.fit_transform(df)
# df_scaler = pd.DataFrame(df)
# df_scaler


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.120970,NaN,NaN,NaN,NaN,NaN,0.379094,0.331178,0.498585,0.402901,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.228569,NaN,NaN,NaN,NaN,NaN,0.387981,0.330116,0.498503,0.401180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.370555,NaN,NaN,NaN,NaN,NaN,0.391664,0.326647,0.497388,0.388614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.149987,NaN,NaN,NaN,NaN,NaN,0.397394,0.329441,0.498202,0.392121,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.228569,NaN,NaN,NaN,NaN,NaN,0.385630,0.322453,0.492338,0.405613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382089,0.073034,0.580969,0.446160,0.611355,0.609088,0.515362,0.356935,0.301263,0.472513,0.428624,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
382090,0.207109,0.616236,0.420826,0.657467,0.614605,0.486393,0.360281,0.293819,0.470631,0.428675,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382091,0.143318,0.555507,0.343269,0.671533,0.611471,0.474024,0.363508,0.323577,0.489632,0.426791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382092,0.283406,0.606141,0.374269,0.628853,0.580657,0.497702,0.365949,0.334901,0.492601,0.426758,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X = df_scaler.values  
# y = responders['responder_6'].values 
y = responders.iloc[inner_index]['responder_6']

# Splitting the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
split_index = int(len(X) * 0.8)  # 前 80% 作為訓練集

# 切分訓練集和測試集
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Impute missing values with the median
median = np.nanmedian(X_train, axis=0)  
X_train = np.where(np.isnan(X_train), median, X_train)
X_test = np.where(np.isnan(X_test), median, X_test)  # Apply medians directly to the test set

X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
X_test = np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)
print(X_train.shape)
print(X_test.shape)

(305675, 128)
(76419, 128)


### 使用tree選變數

In [13]:
lgb_model = LGBMRegressor(random_state=42)
lgb_model.fit(X, y)

symbol_indices = np.where(lgb_model.feature_importances_<20)[0]

# 刪除相應的列
X_train = np.delete(X_train, symbol_indices, axis=1)
X_test = np.delete(X_test, symbol_indices, axis=1)

# 刪除後檢查新形狀
print("X_train 新形狀:", X_train.shape)
print("X_test 新形狀:", X_test.shape)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.232967 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22070
[LightGBM] [Info] Number of data points in the train set: 382094, number of used features: 128
[LightGBM] [Info] Start training from score -0.002164
X_train 新形狀: (305675, 76)
X_test 新形狀: (76419, 76)


In [14]:
use_feature = df.columns[np.where(lgb_model.feature_importances_>=20)[0]]
# symbol_indices

In [15]:
use_feature_lag = [col + "_lag_1" if col.startswith("responder_") else col for col in use_feature]
print(use_feature_lag)


['weight', 'feature_00', 'feature_01', 'feature_02', 'feature_04', 'feature_05', 'feature_06', 'feature_07', 'feature_08', 'feature_13', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_28', 'feature_30', 'feature_32', 'feature_33', 'feature_34', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_68', 'feature_69', 'feature_70', 'feature_71', 'feature_72', 'feature_73', 'feature_74', 'feature_75', 'feature_76', 'feature_77', 'feature_78', 'responder_0_lag_1', 'responder_1_lag_1', 'responder_2_lag_1', 'r

## Modeling

In [16]:
 tscv = TimeSeriesSplit(n_splits=5)
 lasso_cv = LassoCV(alphas=None, cv=tscv, max_iter=10000, random_state=2023)
 linear_lasso = lasso_cv.fit(X_train, y_train)

 print("最好的alpha值:", lasso_cv.alpha_)

最好的alpha值: 3.888539005159383e-05


In [17]:
lasso_model = linear_model.Lasso(alpha=lasso_cv.alpha_, random_state=42)  # Alpha controls the regularization
lasso_model.fit(X_train, y_train)

Lasso(alpha=3.888539005159383e-05, random_state=42)

In [28]:
# LightGBM Hyperparameter Tuning
lgb_params = {
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
    'num_leaves': [31, 63, 127],  # Number of leaves per tree
}
tscv = TimeSeriesSplit(n_splits=5)

lgb_model = LGBMRegressor(random_state=42)
lgb_grid = GridSearchCV(estimator=lgb_model, param_grid=lgb_params, scoring="r2", cv=tscv, n_jobs=-1)
lgb_grid.fit(X_train, y_train)

print("Best LightGBM Parameters:")
print(lgb_grid.best_params_)

# 4. XGBoost Hyperparameter Tuning
xgb_params = {
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of trees
}

xgb_model = XGBRegressor(random_state=42)
xgb_grid = GridSearchCV(estimator=xgb_model, param_grid=xgb_params, scoring="r2", cv=tscv, n_jobs=-1)
xgb_grid.fit(X_train, y_train)

print("\nBest XGBoost Parameters:")
print(xgb_grid.best_params_)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.142509 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19380
[LightGBM] [Info] Number of data points in the train set: 305675, number of used features: 76
[LightGBM] [Info] Start training from score -0.003057
Best LightGBM Parameters:
{'learning_rate': 0.05, 'num_leaves': 31}

Best XGBoost Parameters:
{'learning_rate': 0.05, 'max_depth': 3}


### Evaluation

In [19]:
def mape(y_ture, y_pred):
    return  np.sum(np.abs(y_ture-y_pred) / y_ture) / len(y_ture)


In [20]:
def performance(model, X_test, y_test):
    # testing
    y_test_pred = model.predict(X_test)
    
    r2_test = r2_score(y_test, y_test_pred)
    rmse_test = mean_squared_error(y_test, y_test_pred)**0.5
    mape_test = mape(y_test, y_test_pred)
    test_per = [r2_test, rmse_test, mape_test]
    print("r2_test:", r2_test)
    print("rmse_test:", rmse_test)
    print("mape_test:", mape_test)
    print("")
    
    return test_per

In [29]:
# 3. Model evaluation
performance(lasso_model,X_test, y_test)
performance(lgb_grid,X_test, y_test)
performance(xgb_grid,X_test, y_test)

r2_test: 0.0027293206366119094
rmse_test: 0.8472787865843591
mape_test: 0.04803247246520922

r2_test: 0.004059602793403672
rmse_test: 0.8467134957367545
mape_test: -0.02143373437975161

r2_test: 0.004514720879148393
rmse_test: 0.846520009542832
mape_test: 0.04439326625148032



[0.004514720879148393, 0.846520009542832, 0.04439326625148032]

### Submission

In [22]:
model = xgb_grid

In [23]:
import os
import polars as pl
import kaggle_evaluation.jane_street_inference_server

In [ ]:
import polars as pl
import numpy as np

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags
        # 將 test, lags 合併
        # test_df = test.join(lags_, on=['date_id', 'time_id', 'symbol_id'], how="inner")
        test_df = test.join(lags_, on=['date_id', 'time_id', 'symbol_id'], how="left")
        new_test = test_df.select(use_feature_lag).to_numpy()

    else:
        test_df = test
        missing_features = [col for col in use_feature_lag if col not in test_df.columns]
        for col in missing_features:
            test_df = test_df.with_columns(pl.lit(0).alias(col))
    
    # 選擇所有需要的欄位（存在的欄位和補充的欄位）
    new_test = test_df.select(use_feature_lag).to_numpy()
    
    new_median = np.nanmedian(new_test, axis=0)  
    new_test = np.where(np.isnan(new_test), new_median, new_test)
    new_test = np.nan_to_num(new_test, nan=0.0, posinf=0.0, neginf=0.0)
    print(new_test.shape)

    # 模型預測
    responder_6_predictions = model.predict(new_test)
    # 創建預測結果 DataFrame
    predictions = test.select("row_id").with_columns(
        pl.Series("responder_6", responder_6_predictions)
    )
    print(predictions)
  
    # 打印預測結果中的 row_id
    # print("Predicted row_id:", predictions["row_id"].to_list())
    

    # Ensure the output format and length requirements
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    assert len(predictions) == len(test)
    return predictions

In [25]:
# import traceback

# def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame:
#     """Make a prediction."""
#     try:
#         global lags_
#         if lags is not None:
#             lags_ = lags

#         # 合併 test 和 lags_
#         try:
#             test_df = test.join(lags_, on=['date_id', 'time_id', 'symbol_id'], how="inner")
#         except Exception as e:
#             print("Error during merging test and lags:")
#             traceback.print_exc()
#             raise

#         # 選擇特徵並轉換為 NumPy
#         try:
#             new_test = test_df.select(use_feature_lag).to_numpy()
#             new_median = np.nanmedian(new_test, axis=0)
#             new_test = np.where(np.isnan(new_test), new_median, new_test)
#             new_test = np.nan_to_num(new_test, nan=0.0, posinf=0.0, neginf=0.0)
#         except Exception as e:
#             print("Error during feature selection or transformation:")
#             traceback.print_exc()
#             raise

#         # 模型預測
#         try:
#             responder_6_predictions = model.predict(new_test)
#         except Exception as e:
#             print("Error during model prediction:")
#             traceback.print_exc()
#             raise

#         # 創建預測結果
#         try:
#             predictions = test.select("row_id").with_columns(
#                 pl.Series("responder_6", responder_6_predictions)
#             )
#             predictions = predictions.sort("row_id")
#         except Exception as e:
#             print("Error during result creation:")
#             traceback.print_exc()
#             raise

#         # 確保格式正確
#         assert predictions.columns == ["row_id", "responder_6"]
#         assert len(predictions) == len(test)
#         return predictions
#     except Exception as e:
#         print("Unhandled exception in predict function:")
#         traceback.print_exc()
#         raise


In [26]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

row_id,date_id,time_id,symbol_id,weight,is_scored,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,…,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
i64,i16,i16,i8,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,0,3.169998,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,…,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0,-0.442215,-0.322407,0.143594,-0.92689,-0.782236,-0.036595,-1.305746,-0.795677,-0.143724
1,0,0,1,2.165993,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,…,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0,-0.651829,-1.70784,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
2,0,0,2,3.06555,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,…,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0,-0.656373,-0.264575,-0.892879,-1.511886,-1.03348,-0.378265,-1.57429,-1.863071,-0.027343
3,0,0,3,2.698642,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,-0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0,-0.188186,-0.19097,-0.70149,0.098453,-1.015506,-0.054984,0.329152,-0.965471,0.576635
4,0,0,4,1.80333,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0,-0.257462,-0.471325,-0.29742,0.074018,-0.324194,-0.597093,0.219856,-0.276356,-0.90479
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
34,0,0,34,3.240565,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,…,-0.0,null,null,-0.0,null,-0.0,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,0.0,0.0,-0.185392,-0.187891,-0.206658,-0.634903,-0.643175,-0.443875,-0.556474,-1.122211,-0.884185
35,0,0,35,1.057221,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,…,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,-0.0,-0.0,-0.308923,-0.434147,-1.354941,0.30054,-0.830827,0.424937,0.518839,-0.687369,1.440577
36,0,0,3

(39, 76)
[-0.5079676  -0.5079676  -0.5079676  -0.42246714 -0.5079676  -0.5079676
 -0.4286411  -0.5079676  -0.51980335 -0.5079676  -0.5079676  -0.5079676
 -0.40153378 -0.5079676  -0.51980335 -0.43430296 -0.5079676  -0.4133696
 -0.4133696  -0.5079676  -0.51980335 -0.5079676  -0.40153378 -0.4133696
 -0.51980335 -0.5079676  -0.42246714 -0.4133696  -0.5079676  -0.5079676
 -0.5079676  -0.4133696  -0.51980335 -0.5079676  -0.5079676  -0.41680533
 -0.5079676  -0.5079676  -0.51980335]
shape: (39, 2)
┌────────┬─────────────┐
│ row_id ┆ responder_6 │
│ ---    ┆ ---         │
│ i64    ┆ f32         │
╞════════╪═════════════╡
│ 0      ┆ -0.507968   │
│ 1      ┆ -0.507968   │
│ 2      ┆ -0.507968   │
│ 3      ┆ -0.422467   │
│ 4      ┆ -0.507968   │
│ …      ┆ …           │
│ 34     ┆ -0.507968   │
│ 35     ┆ -0.416805   │
│ 36     ┆ -0.507968   │
│ 37     ┆ -0.507968   │
│ 38     ┆ -0.519803   │
└────────┴─────────────┘
